<a href="https://colab.research.google.com/github/kenpurin/Retrosynthesis_Pytorch/blob/main/score_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kenpurin/Retrosynthesis_Pytorch.git

Cloning into 'Retrosynthesis_Pytorch'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 311 (delta 13), reused 2 (delta 2), pack-reused 285
Receiving objects: 100% (311/311), 14.74 MiB | 15.23 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [2]:
!pip -q install rdkit-pypi

     |████████████████████████████████| 18.6 MB 1.4 MB/s 


In [3]:
from rdkit import Chem
import pandas as pd

In [4]:
invalid_smiles_t= True
beam_size=10
#opt_predictions='/content/predictions_USPTO-50K_model_full_version_step_500000_beam10.txt'
opt_targets='/content/Retrosynthesis_Pytorch/USPTO-50K/tgt/tgt-test.txt'
opt_predictions="/content/Retrosynthesis_Pytorch/prediction/predictions_test.txt"

In [5]:
def canonicalize_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return Chem.MolToSmiles(mol, isomericSmiles=True)
    else:
        return ''

In [6]:
def get_rank(row, base, max_rank):
  for i in range(1, max_rank+1):
    #print(row['{}{}'.format(base, i)])
    if row['target'] == row['{}{}'.format(base, i)]:
      return i
  return 0

In [7]:
with open(opt_targets, 'r') as f:
    targets = [''.join(line.strip().split(' ')) for line in f.readlines()]
targets = targets[:]
predictions = [[] for i in range(beam_size)]
test_df = pd.DataFrame(targets)
test_df.columns = ['target']
total = len(test_df)
total

5004

In [8]:
with open(opt_predictions, 'r') as f:
  for i, line in enumerate(f.readlines()):
    predictions[i % beam_size].append(''.join(line.strip().split(' ')))

In [9]:
for i, preds in enumerate(predictions):
    test_df['prediction_{}'.format(i + 1)] = preds
    test_df['canonical_prediction_{}'.format(i + 1)] = test_df['prediction_{}'.format(i + 1)].apply(
    lambda x: canonicalize_smiles(x))  

In [10]:
test_df

,target,prediction_1,canonical_prediction_1,prediction_2,canonical_prediction_2,prediction_3,canonical_prediction_3,prediction_4,canonical_prediction_4,prediction_5,canonical_prediction_5,prediction_6,canonical_prediction_6,prediction_7,canonical_prediction_7,prediction_8,canonical_prediction_8,prediction_9,canonical_prediction_9,prediction_10,canonical_prediction_10
0,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,,C1=COCCC1.COC(=O)Cc1ccc(O)cc1O,C1=COCCC1.COC(=O)Cc1ccc(O)cc1O,C1=COCCC1.COC(=O)CCc1ccc(O)cc1O,C1=COCCC1.COC(=O)CCc1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1cc(O)cc1O,,C1=COCCC1.COC(=O)CCC(=O)c1cc1O,,C1=COCCC1.COC(=O)CCC(=O)c1cc(O)ccc1O,C1=COCCC1.COC(=O)CCC(=O)c1cc(O)ccc1O,rC1CCCCO1.COC(=O)CCC(=O)c1ccc(O)cc1O,,C1=CoCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1OC,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1OC
1,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ncccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cnccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccncc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccncc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccncc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1c(N)cccn1,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccN)cn1,,COC(=O)c1cc(C(=O)O)cc1.Nc1cccnc1N,,COC(=O)c1ccc2c(C(=O)O)c1.Nc1cccnc1N,,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N.O,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N.O
2,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)Cc1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CONC.CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CC=C(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CC=C(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)OC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)OC1.CNOC,C(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CNOC.CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC),,CC(C)(C)OC(=O)NC1CCC(C(=O)O)(CC)C)CC1,
3,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1ccC(Cl)nc1Cl,,Nc1ccC(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,,Nc1ccc(O)cc1.O=[N+]([O-])c1cccCl)nc1Cl,,Nc1ccc(O)cc1.O=[N+]([O-])c1cc(Cl)nc1Cl,,Nc1ccc(O)cc1.O=[N+]([O-])c1cccc(Cl)nc1Cl,,Nc1ccc(O)cC1.O=[N+]([O-])c1ccc(Cl)nc1Cl,,Nc1c(O)ccc(O)c1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1cc(O)ccc1O.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1cc(O)ccc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1cccc(O)c1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1ClCl,
4,[N-]=[N+]=NCC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([...,N.O=[N+]([O-])[C@H]1CC(CBr)=CC[C@@H]1c1ccc(Cl)...,N.O=[N+]([O-])[C@H]1CC(CBr)=CC[C@@H]1c1ccc(Cl)...,N.O=[N+]([O-])[C@@H]1CC(CN=[N+]=[N-])=CC[C@@H]...,N.[N-]=[N+]=NCC1=CC[C@H](c2ccc(Cl)cc2Cl)[C@H](...,[N-]=[N+]=NCC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([...,[N-]=[N+]=NCC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([...,N.O=[N+]([O-])C1CC(CBr)=CC[C@@H]1c1ccc(Cl)cc1Cl,N.O=[N+]([O-])C1CC(CBr)=CC[C@@H]1c1ccc(Cl)cc1Cl,N.O=[N+]([O-])[C@H]1CC(CCl)=CC[C@@H]1c1ccc(Cl)...,N.O=[N+]([O-])[C@H]1CC(CCl)=CC[C@@H]1c1ccc(Cl)...,N.O=[N+]([O-])[C@@H]1CC(CBr)=CC[C@@H]1c1ccc(Cl...,N.O=[N+]([O-])[C@@H]1CC(CBr)=CC[C@@H]1c1ccc(Cl...,N.O=[N+]([O-])[C@@H]1CC(CN=[N+]=[N-])=CC[C@H]1...,N.[N-]=[N+]=NCC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]...,N.O=[N+]([O-])[C@@H]1CC(CN)=CC[C@@H]1c1ccc(Cl)...,N.NCC1=CC[C@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N.O=[N+]([O-])[C@@H]1CC(CNC(=O)C2=CC[C@@H](c3c...,N.O=C(NCC1CC([N+](=O)[O-])C1)C1=CC[C@@H](c2ccc...,N.O=[N+]([O-])[C@@H]1CC(CBr)=CC[C@@H]1c1ccc(Cl...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999,Cc1cc(

In [11]:
test_df['rank'] = test_df.apply(lambda row: get_rank(row, 'canonical_prediction_', beam_size), axis=1)
test_df['rank']

0       1
1       1
2       1
3       1
4       3
       ..
4999    4
5000    0
5001    1
5002    0
5003    0
Name: rank, Length: 5004, dtype: int64

In [12]:
correct = 0
invalid_smiles = 0

for i in range(1, beam_size+1):
    correct += (test_df['rank'] == i).sum()
    print((test_df['rank'] == i).sum(),correct)
    
    invalid_smiles += (test_df['canonical_prediction_{}'.format(i)] == '').sum()
    if invalid_smiles_t:
        print('Top-{}: {:.1f}% || Invalid SMILES {:.2f}%'.format(i, correct/total*100,
                                                                 invalid_smiles/(total*i)*100))
    else:
        print('Top-{}: {:.1f}%'.format(i, correct / total * 100))

2810 2810
Top-1: 56.2% || Invalid SMILES 2.24%
500 3310
Top-2: 66.1% || Invalid SMILES 9.70%
194 3504
Top-3: 70.0% || Invalid SMILES 14.86%
101 3605
Top-4: 72.0% || Invalid SMILES 18.75%
57 3662
Top-5: 73.2% || Invalid SMILES 21.81%
50 3712
Top-6: 74.2% || Invalid SMILES 24.26%
32 3744
Top-7: 74.8% || Invalid SMILES 26.29%
19 3763
Top-8: 75.2% || Invalid SMILES 28.32%
14 3777
Top-9: 75.5% || Invalid SMILES 30.54%
10 3787
Top-10: 75.7% || Invalid SMILES 33.20%
